In [1]:
#! pip install transformers torch
 

#### This notbook is used for getting score for from distilbert-base-uncased-finetuned-sst-2 
* depends on each sentence or each scene 

In [2]:
from transformers import pipeline
import re
import os
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
import torch
import pandas as pd
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\an_na\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:

def remove_ext_int_lines(text):
    # Remove all lines that include EXT or INT
    lines = text.split('\n')
    cleaned_lines = [line for line in lines if not line.strip().startswith(('EXT', 'INT'))]
    cleaned_text = '\n'.join(cleaned_lines)
    return cleaned_text

# get model and tokenizer 
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = DistilBertTokenizer.from_pretrained(model_name)
model = DistilBertForSequenceClassification.from_pretrained(model_name)
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

# def function to deal with one file
def classify_and_save_scene(input_file, output_file):
    # open file 
    with open(input_file, 'r', encoding='utf-8') as f:
        text_0 = f.read()
    text = remove_ext_int_lines(text_0)
    scenes = text.split("==================================================")
    num_scenes = len(scenes)

    results = []
    for scene in scenes:
        inputs = tokenizer(scene, return_tensors="pt", truncation=True, padding=True, max_length=512)
        with torch.no_grad():
            logits = model(**inputs).logits
            predicted_class_id = logits.argmax().item()
            label = model.config.id2label[predicted_class_id]
            score = torch.nn.functional.softmax(logits, dim=1).max().item()
            results.append({"label": label, "score": score})
    
    # write the results into new_file
    with open(output_file, 'w') as f:
        for sentence, result in zip(scenes, results):
            f.write(f"Sentence: {sentence}\n")
            f.write(f"Prediction: {result['label']} (score: {result['score']})\n\n")

    output_to_csv = output_file.replace('.txt','.csv')
    df = pd.DataFrame(results)
    df.to_csv(output_to_csv, index=False)

    #print(f"Results saved to {output_file}")

# test
#input_file = "/Users/xiaozhouye/Desktop/neuefische/Capstone-project_copy/data/screenplay_data/data/scene_separated_texts/3 Idiots_1187043.txt"
#output_file = "/Users/xiaozhouye/Desktop/neuefische/Capstone-project_copy/data/screenplay_data/data/scene_separated_analysed/3 Idiots_1187043.txt"


#classify_and_save_scene(input_file, output_file)




c:\Users\an_na\Documents\Arbeit\Data Science Weiterbildung\capstone\Capstone-project\.venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

c:\Users\an_na\Documents\Arbeit\Data Science Weiterbildung\capstone\Capstone-project\.venv\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\an_na\.cache\huggingface\hub\models--distilbert-base-uncased-finetuned-sst-2-english. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [4]:
import pandas as pd
import os
try:
    directory_path = 'data/screenplay_data/data/scene_separated_texts'#---------replace your path
    files = os.listdir(directory_path)
    directory_path_new = 'data/screenplay_data/data/scene_separated_sentiment'#---------replace your path
    if not os.path.exists(directory_path_new):
        os.makedirs(directory_path_new)
    #print(len(files))
    for file in files:
        file_old = os.path.join(directory_path,file)
        file_new = os.path.join(directory_path_new,file)
        classify_and_save_scene(file_old, file_new)
except Exception as e:
    print(f"Error occurred: {e}")

Error occurred: 'charmap' codec can't encode character '\x8a' in position 5674: character maps to <undefined>
